In [7]:
import os
os.chdir( "/home/lucasfuzato/CODING/WINE" )

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from src.aux_fun import *

tests_df = pd.DataFrame( columns = [ "tree_num" , "num_features" ] )

tree_size = [ 5 , 10 , 50 , 100 ]
feature_select = [ "log2" , "sqrt"]
for i , ( tree_num , num_features ) in enumerate( product( tree_size , feature_select ) ):
    tests_df.loc[ i ] = pd.Series( [ tree_num , num_features ] , [ "tree_num" , "num_features" ] )
print( tests_df ) 

  tree_num num_features
0        5         log2
1        5         sqrt
2       10         log2
3       10         sqrt
4       50         log2
5       50         sqrt
6      100         log2
7      100         sqrt


In [9]:
from sklearn.ensemble import RandomForestClassifier as rd_frst
seed_val = 47

depth , algo = 25 , "entropy"

X , y , X_2d , columns = class_get_data()

scores = [ "f1_score" , "precision" , "recall" ]
metrics = [ "mean" , "std" ]
data = { tup:list() for tup in product( scores , metrics ) }

for simu_id , ( tree_num , num_features ) in enumerate( product( tree_size , feature_select ) ):

    lst = []
    fold_iter = get_prepared_data( X , y , seed = seed_val , smote = False )
    for ( X_train , y_train ) , ( X_test , y_test ) in fold_iter:

        forest = rd_frst( n_estimators = tree_num , max_features = num_features, criterion = algo , max_depth = depth ,  )
        forest.fit( X_train , y_train )

        ts_cls = forest.predict( X_test )
        ts_f1 , prec , recall = bin_f1_score( ts_cls , y_test )

        lst.append( ( ts_f1 , prec , recall ) )
    
    arr = np.array( lst )
    means = arr.mean( axis = 0 )
    data[ ( "f1_score", "mean" ) ].append( means[ 0 ] )
    data[ ( "precision", "mean" ) ].append( means[ 1 ] )
    data[ ( "recall", "mean" ) ].append( means[ 2 ] )

    stds  = arr.std( axis = 0 )
    data[ ( "f1_score", "std" ) ].append( stds[ 0 ] )
    data[ ( "precision", "std" ) ].append( stds[ 1 ] )
    data[ ( "recall", "std" ) ].append( stds[ 2 ] )

print( pd.DataFrame( data ) )
print()

   f1_score           precision              recall          
       mean       std      mean       std      mean       std
0  0.645888  0.043365  0.720565  0.034465  0.586639  0.055495
1  0.634203  0.030771  0.703802  0.036907  0.578771  0.040624
2  0.667520  0.036204  0.817392  0.035436  0.565422  0.043618
3  0.655442  0.026008  0.786675  0.037588  0.563866  0.040460
4  0.691491  0.022005  0.817582  0.030160  0.599883  0.027539
5  0.686615  0.033498  0.820561  0.047182  0.591271  0.034064
6  0.695682  0.024194  0.832222  0.044329  0.599090  0.029658
7  0.693958  0.021058  0.825133  0.033272  0.599865  0.028240

